In [73]:
import pandas as pd
import numpy as np
import random
import math
from pandas import DataFrame
from scipy import stats

In [2]:
#link1과 15데이터를 1and15.txt에 저장

In [3]:
f = open("1and15.txt",'w')

In [4]:
df = open('용인_양지.txt', 'r')
lines = df.readlines()
for line in lines:
    if not line: break
    result = line.split(";")
    if(result[2] == '15' or result[2] == '1'):
        f.write(';'.join(result))
df.close()

In [5]:
f.close()

In [6]:
#1and15(링크1과 링크 15데이터)에 있는 NO데이터를 1and15_NO에 저장 ->이 차량들의 모두가 대상차량은 아니지만, 여기에 속하지 않는 것은 대상차량이 될 수 없음

In [7]:
f = open("1and15_NO.txt",'w')

df = open("1and15.txt",'r')
lines = df.readlines()

for line in lines:
    if not line : break
    result = line.split(";")
    f.write(result[1])
    f.write('\n')
    
df.close()
f.close()

In [8]:
#바로 위에서 저장한 파일을 NO_data로 불러오기

In [9]:
NO_data = pd.read_csv('1and15_NO.txt', names=['NO'])

In [10]:
NO_data.head()

,NO
0,728
1,734
2,742
3,746
4,756


In [11]:
NO_data.tail()

,NO
198109,4695
198110,4698
198111,4710
198112,4720
198113,4721


In [12]:
NO_data.shape

(198114, 1)

In [13]:
#중복된 NO제거
NO_data = NO_data.drop_duplicates('NO')

In [14]:
NO_data.head()

,NO
0,728
1,734
2,742
3,746
4,756


In [15]:
NO_data.tail()

,NO
197737,5055
197820,5056
197821,5057
197904,5058
198075,5059


In [16]:
NO_data.shape

(4294, 1)

In [17]:
NO_data = NO_data.set_index('NO')
NO_data["Link1"] = 0
NO_data["Link15"] = 0

In [18]:
NO_data.head()

,Link1,Link15
NO,,
728,0,0
734,0,0
742,0,0
746,0,0
756,0,0


In [19]:
#링크 1 또는 링크 15를 지나면 NO_data 데이터 프레임의 해당 열을 1로 바꿈

In [20]:
f = open("1and15.txt",'r')

lines = f.readlines()
for line in lines:
    if not line: break
    result = line.split(";")
    if(result[2] == '15'):
        ind = int(result[1])
        NO_data.loc[ind]['Link1'] = 1
    if(result[2] == '1'):
        ind = int(result[1])
        NO_data.loc[ind]['Link15'] = 1

f.close()   

In [21]:
NO_data.head()

,Link1,Link15
NO,,
728,1,0
734,1,0
742,1,0
746,1,0
756,1,0


In [22]:
NO_data.tail()

,Link1,Link15
NO,,
5055,0,1
5056,0,1
5057,0,1
5058,0,1
5059,0,1


In [23]:
# 링크1과 링크 15가 둘 다 1인 NO를 veh_list에 저장
veh_list = []
veh_list = [i for i in NO_data.index if NO_data.loc[i]['Link1']==1 and NO_data.loc[i]['Link15'] == 1]

In [24]:
#veh_list를 데이터프레임 형식으로 변환 후 veh_list.xlsx로 저장
veh_list_df = DataFrame(veh_list)
veh_list_df.to_excel('veh_list.xlsx', sheet_name = 'veh_list') 

In [25]:
veh_list_df.shape

(3578, 1)

In [26]:
#NO_data를 NO_data2로 다시 불러오기 (같은 데이터임) // 중복 삭제 없는 모든 데이터
NO_data2 = pd.read_csv('1and15_NO.txt', names=['NO'])
NO_data2.head()

,NO
0,728
1,734
2,742
3,746
4,756


In [27]:
NO_data2.shape

(198114, 1)

In [28]:
NO_data2 = NO_data2.drop_duplicates('NO')

In [29]:
NO_data2 = DataFrame(NO_data2)
NO_data2 = NO_data2.set_index('NO')

In [30]:
NO_data2.head()

""
NO
728
734
742
746
756


In [31]:
NO_data2.tail()

""
NO
5055
5056
5057
5058
5059


In [32]:
NO_data2.shape

(4294, 0)

In [33]:
#link1의 142m지점과 링크15의 399m지점 지나기 전과 후 pos와 sec저장 위해 NO_data를 인덱스로하여 열 생성
#A가 after B는 Before
NO_data2['LINK1_sec_B'] = None
NO_data2['LINK1_pos_B'] = None
NO_data2['LINK1_sec_A'] = None
NO_data2['LINK1_pos_A'] = None

NO_data2['LINK15_sec_B'] = None
NO_data2['LINK15_pos_B'] = None
NO_data2['LINK15_sec_A'] = None
NO_data2['LINK15_pos_A'] = None

In [34]:
NO_data2.head()

,LINK1_sec_B,LINK1_pos_B,LINK1_sec_A,LINK1_pos_A,LINK15_sec_B,LINK15_pos_B,LINK15_sec_A,LINK15_pos_A
NO,,,,,,,,
728,None,None,None,None,None,None,None,None
734,None,None,None,None,None,None,None,None
742,None,None,None,None,None,None,None,None
746,None,None,None,None,None,None,None,None
756,None,None,None,None,None,None,None,None


In [35]:
#링크 1
#142m를 지나고 A에 데이터가 저장되어 있지 않다면 그 때의 sec와 pos를 저장
#142m전 데이터는 for문으로 반복하면서 B에 계속 저장

In [36]:

df = open("1and15.txt",'r')

lines = df.readlines()

for line in lines:
    if not line : break
    result = line.split(";")
    if(result[2] == '1'):
        no = float(result[1])
        sec = float(result[0])
        pos = float(result[4])
        if(float(result[4]) > 142):
            if((pd.isnull(NO_data2.loc[no,"LINK1_sec_A"])==True) and (pd.isnull(NO_data2.loc[no,"LINK1_pos_A"]) == True)):                
                NO_data2.loc[no,"LINK1_sec_A"] = sec
                NO_data2.loc[no,"LINK1_pos_A"] = pos
        
        elif(float(result[4]) <= 142):            
            NO_data2.loc[no,"LINK1_sec_B"] = sec
            NO_data2.loc[no,"LINK1_pos_B"] = pos
            

df.close()            
        
    

In [37]:
#링크 1의 방법과 같음

In [38]:
df = open("1and15.txt",'r')

lines = df.readlines()

for line in lines:
    if not line : break
    result = line.split(";")
    if(result[2] == '15'):
        no = float(result[1])
        sec = float(result[0])
        pos = float(result[4])
        if(float(result[4]) > 399):
            if(pd.isnull(NO_data2.loc[no,"LINK15_sec_A"])==True and pd.isnull(NO_data2.loc[no,"LINK15_pos_A"]) == True):                
                NO_data2.loc[no,"LINK15_sec_A"] = sec
                NO_data2.loc[no,"LINK15_pos_A"] = pos
        
        elif(float(result[4]) <= 399):            
            NO_data2.loc[no,"LINK15_sec_B"] = sec
            NO_data2.loc[no,"LINK15_pos_B"] = pos
            

df.close()            

In [40]:
NO_data2.to_excel('NO_data2.xlsx')

In [41]:
#위 과정을 거쳐서 데이터가 저장되지 않았다면, 링크1과 링크15를 모두 지난 차량이라고 할 수 없으므로 제거
#ps.NO_data는 단순히 링크1과 링크15가 있는 데이터의 차량을 저장한 것
NO_data3 = NO_data2.dropna(axis=0)

In [42]:
NO_data3.shape

(3518, 8)

In [43]:
NO_data3.head()

,LINK1_sec_B,LINK1_pos_B,LINK1_sec_A,LINK1_pos_A,LINK15_sec_B,LINK15_pos_B,LINK15_sec_A,LINK15_pos_A
NO,,,,,,,,
1141,902,125.34,903,150.13,1228,387.32,1229,412.11
1142,902,117.86,903,146.88,1193,372.58,1194,400.23
1143,902,114.74,903,145.15,1183,388.9,1184,419.31
1144,903,139.68,904,169.16,1194,383.27,1195,412.75
1145,905,137.92,906,162.81,1232,383.13,1233,408.02


In [44]:
#이를 1and15_BA.xlsx로 저장
NO_data3.to_excel('1and15_BA.xlsx', sheet_name = 'data')

In [45]:
#다시 불러옴
NO_data4 = pd.read_excel('1and15_BA.xlsx')

In [46]:
#링크1을 지났을 때의 시간과 링크15를 지났을 때의 시간을 저장하기 위해 열 생성

In [47]:
NO_data4['t1'] = 0
NO_data4['t15'] = 0

In [48]:
#보간법으로 용인ic 지점과 양지 ic지점 지났을 때 시간 계산하여 저장
for i in NO_data4.index:
    NO_data4.loc[i, "t1"] = NO_data4.loc[i,"LINK1_sec_B"] + 142 / (NO_data4.loc[i, "LINK1_pos_A"] - NO_data4.loc[i, "LINK1_pos_B"])
    NO_data4.loc[i, "t15"] = NO_data4.loc[i,"LINK15_sec_B"] + 399 / (NO_data4.loc[i, "LINK15_pos_A"] - NO_data4.loc[i, "LINK15_pos_B"])

In [49]:
NO_data4.to_excel('1and15_BA.xlsx', sheet_name = 'data')

In [50]:
#Travel Time 저장위해 TT열 생성, t15 - t1값을 TT에 저장

In [51]:
NO_data4['TT'] = 0

for i in NO_data4.index:
    NO_data4.loc[i, "TT"] = NO_data4.loc[i,"t15"] - NO_data4.loc[i,"t1"]

In [52]:
NO_data4.to_excel('1and15_BA.xlsx', sheet_name = 'data')

In [53]:
NO_data5 = pd.read_excel('1and15_BA.xlsx')

In [54]:
#TT를 제외한 열은 사용하지 않으므로 TT열만 추출

In [55]:
TT = NO_data5["TT"]

In [56]:
TT.head()

0    336.367084
1    300.537203
2    289.451167
3    299.717775
4    337.325432
Name: TT, dtype: float64

In [57]:
size = len(TT)

In [80]:
length = 8130.697

In [58]:
TT2 = DataFrame(TT)

In [59]:
TT2.head()

,TT
0,336.367084
1,300.537203
2,289.451167
3,299.717775
4,337.325432


In [60]:
TT2['TMS'] = (8130.679*3600)/(TT2['TT']*1000)

In [61]:
TT2.head()

,TT,TMS
0,336.367084,87.019348
1,300.537203,97.393747
2,289.451167,101.123947
3,299.717775,97.660022
4,337.325432,86.772125


In [ ]:
#표준오차

In [72]:
iter = 0
f = open('1_rand_SE.txt', 'w')

while(iter!=10):
    arr = np.random.uniform(low=0.0, high = 1.0, size=size)
    arr = arr.tolist()
    df = pd.DataFrame(arr)
    df.columns = ['RAND']
    rand = pd.merge(TT2, df, how = 'outer', left_index = True, right_index = True)
    rand1 = []
    rand1 = [rand.loc[i,"TT"] for i in rand.index if rand.loc[i,"RAND"] <= 0.01]
    s = np.std(rand1) 
    n = len(rand1)
    SE = s/math.sqrt(n)
    f.write(str(SE))
    f.write('\n')
    iter = iter + 1
    
f.close()
    
    

In [74]:
n

32

In [75]:
rand.head()

,TT,TMS,RAND
0,336.367084,87.019348,0.996098
1,300.537203,97.393747,0.740326
2,289.451167,101.123947,0.158453
3,299.717775,97.660022,0.903810
4,337.325432,86.772125,0.581730


In [79]:
rand1

[308.8531787398997,
 285.4200548336628,
 302.9985994397759,
 296.2990713239005,
 348.8210526315788,
 342.3248074195612,
 362.130359462971,
 368.5298340062809,
 307.625468164794,
 298.2700353551149,
 305.5031005235992,
 333.4769669792092,
 292.9560992730933,
 348.7027083054527,
 298.2497689505824,
 307.2884217740493,
 341.224571083867,
 288.6473599183869,
 295.9731973341368,
 330.3903546377273,
 287.5586005505538,
 292.8997289972899,
 315.7512747475357,
 283.8067539456429,
 297.640459340023,
 341.6720070646406,
 366.0754482627071,
 282.8887140691568,
 315.1240290030933,
 300.0602930823898,
 281.7037369148766,
 316.1194994285906]

In [78]:
sum100 = rand['TT'].sum(axis=0)
sum100

1088608.4409344252

In [83]:
sms100 = (length*size*3.6)/sum100
sms100

94.59200158067014

In [82]:
#http://thenotes.tistory.com/entry/Ttest-in-python
tTest1 = stats.ttest_1samp(rand1,sms100)
tTest1

Ttest_1sampResult(statistic=47.37965887382028, pvalue=1.7218170321084378e-30)

In [84]:
rand2 = []
rand2 = [rand.loc[i,"TT"] for i in rand.index if rand.loc[i,"RAND"] <= 0.02]

tTest2 = stats.ttest_1samp(rand2, sms100)
tTest2

Ttest_1sampResult(statistic=73.35897616999152, pvalue=2.0802661681410658e-69)

In [85]:
rand3 = []
rand3 = [rand.loc[i,"TT"] for i in rand.index if rand.loc[i,"RAND"] <= 0.03]

tTest3 = stats.ttest_1samp(rand3, sms100)
tTest3

Ttest_1sampResult(statistic=82.51930741203581, pvalue=6.573228237707417e-96)

In [87]:
rand4 = []
rand4 = [rand.loc[i,"TT"] for i in rand.index if rand.loc[i,"RAND"] <= 0.04]

tTest4 = stats.ttest_1samp(rand4, sms100)
tTest4

Ttest_1sampResult(statistic=96.79679555156383, pvalue=1.2291297403345672e-133)

In [88]:
rand5 = []
rand5 = [rand.loc[i,"TT"] for i in rand.index if rand.loc[i,"RAND"] <= 0.05]

tTest5 = stats.ttest_1samp(rand5, sms100)
tTest5

Ttest_1sampResult(statistic=110.31387867890658, pvalue=2.472589083953744e-169)

In [ ]:
arr = np.random.uniform(low=0.0, high = 1.0, size = size)

In [ ]:
arr = arr.tolist()

In [ ]:
df = pd.DataFrame(arr)

In [ ]:
df.head()

In [ ]:
df.columns = ['RAND']

In [ ]:
df.head()

In [ ]:
TT2.head()

In [ ]:
#TT와 arr을 dataframe으로 변경하여 merge

In [ ]:
rand = pd.merge(TT2, df, how = 'outer', left_index = True, right_index = True)

In [ ]:
rand.to_excel('Rand.xlsx')

In [ ]:
np.savetxt('Rand.txt', rand, fmt='%d',delimiter = ';', header = "TT\Rand")

In [ ]:
rand = pd.read_excel('Rand.xlsx')

In [ ]:
rand.head()

In [ ]:
rand['TT'].sum()

In [ ]:
#용인IC와 양지IC의 거리 8130.679m, 차량대수 4345대
#비율에 따라 계산

In [ ]:
#100%
sms100 = (8130.679* 4345*3.6)/rand['TT'].sum()
sms100

In [ ]:
#1%
n = 0;
sum = 0;
for i in rand.index:
    if rand.loc[i,"RAND"]<=0.01:
        sum += rand.loc[i,"TT"]
        n = n+1
        
sms1 = (8130.679*n*3.6)/sum
sms1     
    

In [ ]:
#2%
n = 0;
sum = 0;
for i in rand.index:
    if rand.loc[i,"RAND"]<=0.02:
        sum += rand.loc[i,"TT"]
        n = n+1
        
sms2 = (8130.679*n*3.6)/sum
sms2     
    

In [ ]:
n = 0;
sum = 0;
for i in rand.index:
    if rand.loc[i,"RAND"]<=0.03:
        sum += rand.loc[i,"TT"]
        n = n+1
        
sms3 = (8130.679*n*3.6)/sum
sms3 

In [ ]:

n = 0;
sum = 0;
for i in rand.index:
    if rand.loc[i,"RAND"]<=0.04:
        sum += rand.loc[i,"TT"]
        n = n+1
        
sms4 = (8130.679*n*3.6)/sum
sms4     
    

In [ ]:

n = 0;
sum = 0;
for i in rand.index:
    if rand.loc[i,"RAND"]<=0.05:
        sum += rand.loc[i,"TT"]
        n = n+1
        
sms5 = (8130.679*n*3.6)/sum
sms5     
    

In [ ]:

n = 0;
sum = 0;
for i in rand.index:
    if rand.loc[i,"RAND"]<=0.1:
        sum += rand.loc[i,"TT"]
        n = n+1
        
sms10 = (8130.679*n*3.6)/sum
sms10     
    

In [ ]:

n = 0;
sum = 0;
for i in rand.index:
    if rand.loc[i,"RAND"]<=0.2:
        sum += rand.loc[i,"TT"]
        n = n+1
        
sms20 = (8130.679*n*3.6)/sum
sms20     
    

In [ ]:

n = 0;
sum = 0;
for i in rand.index:
    if rand.loc[i,"RAND"]<=0.25:
        sum += rand.loc[i,"TT"]
        n = n+1
        
sms25 = (8130.679*n*3.6)/sum
sms25     
    

In [ ]:
n = 0;
sum = 0;
for i in rand.index:
    if rand.loc[i,"RAND"]<=0.3:
        sum += rand.loc[i,"TT"]
        n = n+1
        
sms30 = (8130.679*n*3.6)/sum
sms30

In [ ]:
#50%
n = 0;
sum = 0;
for i in rand.index:
    if rand.loc[i,"RAND"]<=0.5:
        sum += rand.loc[i,"TT"]
        n = n+1
        
sms50 = (8130.679*n*3.6)/sum
sms50

In [ ]:
#70%
n = 0;
sum = 0;
for i in rand.index:
    if rand.loc[i,"RAND"]<=0.7:
        sum += rand.loc[i,"TT"]
        n = n+1
        
sms70 = (8130.679*n*3.6)/sum
sms70     
    

In [ ]:
#90%
n = 0;
sum = 0;
for i in rand.index:
    if rand.loc[i,"RAND"]<=0.9:
        sum += rand.loc[i,"TT"]
        n = n+1
        
sms90 = (8130.679*n*3.6)/sum
sms90     
    